In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
coup_curr_ver = pd.read_csv("coups-merged.csv")
p5_df = pd.read_excel("p5v2018.xls")
poly_vio_df = pd.read_csv("MEPVv2018.csv")

p5_df.head(5)
print(poly_vio_df.columns)

Index(['scode', 'ccode', 'country', 'year', 'ind', 'intind', 'intviol',
       'intwar', 'civviol', 'civwar', 'ethviol', 'ethwar', 'inttot', 'civtot',
       'actotal', 'nborder', 'region', 'nregion'],
      dtype='object')


In [3]:
#Rename columns from polity-5 dataset
#Dropped duplicate region column and old GDP colum
#Neighbors w/ violence vs. regional w/ violence (Do we need both?)
#


coup_curr_ver = coup_curr_ver.rename(columns={"INTIND": "indep_viol_mag", "INTVIOL":"intl_viol_mag", "INTWAR":"intl_war_mag", 
                        "CIVVIOL":"civil_viol_mag", "CIVWAR":"civil_war_mag", "ETHVIOL":"ethnic_viol_mag",
                        "ETHWAR":"ethnic_war_mag", "TOTINT":"total_intl_viol_mag", "TOTCIV":"total_civil_viol_mag",
                        "nINT": "neighbors_w_intl_viol", "nCIV":"neighbors_w_civil_viol", "nREGION":"neighbors_in_region",
                        "REGINT":"regional_intl_viol_mag", "REGCIV": "regional_civil_viol_mag", "REGAC":"total_regional_viol_mag",
                        "nRINT":"reg_states_intl_viol_count", "nRCIV":"reg_states_civil_viol_count"})

coup_curr_ver = coup_curr_ver.rename(columns={'democ': 'democracy', 'autoc': 'autocracy',
                        'polity2':'polity2_score', 'durable':'years_of_regime_pow', 'xrcomp':'exec_competitive_score', 
                        'xropen': 'exec_openness_score', 'xconst': 'exec_constraint_score', 'parreg' : 'political_regularity', 
                        'parcomp' : 'political_competitiveness','exrec': 'exec_recruitment'})

coup_curr_ver = coup_curr_ver.drop(columns=["REGION", "GDP per capita"])

print(coup_curr_ver.columns)

coup_curr_ver.to_csv("coup_merged_v2.csv")

Index(['Unnamed: 0.1', 'Unnamed: 0', 'coup_id', 'cowcode', 'country', 'year',
       'month', 'day', 'event_type', 'unrealized', 'realized', 'conspiracy',
       'attempt', 'military', 'dissident', 'rebel', 'palace', 'foreign',
       'auto', 'resign', 'popular', 'counter', 'other', 'noharm', 'injured',
       'killed', 'harrest', 'jailed', 'tried', 'fled', 'exile', 'region',
       'sub-region', 'gini', 'indep_viol_mag', 'intl_viol_mag', 'intl_war_mag',
       'civil_viol_mag', 'civil_war_mag', 'ethnic_viol_mag', 'ethnic_war_mag',
       'total_intl_viol_mag', 'total_civil_viol_mag', 'neighbors_w_intl_viol',
       'neighbors_w_civil_viol', 'neighbors_in_region',
       'regional_intl_viol_mag', 'regional_civil_viol_mag',
       'total_regional_viol_mag', 'reg_states_intl_viol_count',
       'reg_states_civil_viol_count', 'democracy', 'autocracy',
       'polity2_score', 'years_of_regime_pow', 'exec_competitive_score',
       'exec_openness_score', 'exec_constraint_score', 'political_

In [4]:
x = coup_curr_ver.isna().sum(axis=0)
x[x > 0]

'''Based on guide, for polity score: 
-Cases of -77 are to be treated as 0 (and perhaps we add another column),
-Cases of -88 are to have the differences spreas across them:

    For example, country X has a POLITY score of -7 in 1957, followed
    by three years of -88 and, finally, a score of +5 in 1961. The change
    (+12) would be prorated over the intervening three years at a rate of
    per year, so that the converted scores would be as follows: 1957 -7;
    1958 -4; 1959 -1; 1960 +2; and 1961 +5.

-Cases of -66 are true missingness (Page 17 of Polity5 Guide)
'''
country_ids = ['coup_id', 'cowcode', 'country', 'year']
polity_old_cols = ['democ', 'autoc', 'polity', 'polity2', 'durable', 'xrreg', 'xrcomp',
       'xropen', 'xconst', 'parreg', 'parcomp', 'exrec']
polity_imputation_cols = ['democracy', 'autocracy',
       'polity2_score', 'years_of_regime_pow', 'exec_competitive_score',
       'exec_openness_score', 'exec_constraint_score', 'political_regularity',
       'political_competitiveness', 'exec_recruitment']

coup_pol_impute = coup_curr_ver[country_ids + polity_imputation_cols]


# Impute 0s for -77 and create interregum (b/t governments) column 
display(coup_pol_impute.head(5))

coup_curr_ver["standing_regime"] = np.where(coup_curr_ver["democracy"] == -77, 0, 1)
for feature in polity_imputation_cols:
    coup_curr_ver[feature].mask(coup_curr_ver[feature] == -77, 0, inplace=True)
display(coup_curr_ver)

#Impute 

coup_curr_ver.to_csv("-77_yeeted.csv")



interregum_coup = coup_pol_impute[coup_pol_impute['democracy'] == -88]
display(interregum_coup)

interregum_coup['country'].value_counts()


coup_id  cowcode       country  year  democracy  autocracy  \
0  43910302014      439  Burkina Faso  2014        2.0        2.0   
1  51605132015      516       Burundi  2015        2.0        3.0   
2  48304132006      483          Chad  2006        1.0        3.0   
3  65102112011      651         Egypt  2011        0.0        2.0   
4  65107032013      651         Egypt  2013        0.0        4.0   

   polity2_score  years_of_regime_pow  exec_competitive_score  \
0            0.0                 13.0                     0.0   
1           -1.0                  0.0                     1.0   
2           -2.0                 14.0                     0.0   
3           -2.0                  6.0                     0.0   
4           -4.0                  0.0                     0.0   

   exec_openness_score  exec_constraint_score  political_regularity  \
0                  0.0                    3.0                   3.0   
1                  4.0                    4.0                   3.0   
2                  0.0                    2.0                   3.0   
3                  0.0                    2.0                   2.0   
4                  0.0                    3.0                   4.0   

   political_competitiveness  exec_recruitment  
0                        4.0               5.0  
1                        3.0               3.0  
2                        3.0               5.0  
3                        0.0               5.0  
4                        2.0               5.0

Unnamed: 0.1  Unnamed: 0      coup_id  cowcode       country  year  \
0               0           0  43910302014      439  Burkina Faso  2014   
1               1           1  51605132015      516       Burundi  2015   
2               2           2  48304132006      483          Chad  2006   
3               3           3  65102112011      651         Egypt  2011   
4               4           4  65107032013      651         Egypt  2013   
..            ...         ...          ...      ...           ...   ...   
938           938         938  55106141981      551        Zambia  1981   
939           939         939  55110071988      551        Zambia  1988   
940           940         940  55106301990      551        Zambia  1990   
941           941         941  55110281997      551        Zambia  1997   
942           942         942  55201111999      552      Zimbabwe  1999   

     month  day  event_type  unrealized  ...    gdppc  gdpgrwth  cntry_change  \
0       10   30        coup           0  ...   1930.0      8.06             0   
1        5   13   attempted           1  ...    795.0      3.91             0   
2        4   13   attempted           1  ...   1570.0     0.528             0   
3        2   11        coup           0  ...  10300.0      1.33             0   
4        7    3        coup           0  ...  10400.0      1.79             0   
..     ...  ...         ...         ...  ...      ...       ...           ...   
938      6   14  conspiracy           1  ...   2180.0      2.48             0   
939     10    7  conspiracy           1  ...   2050.0     0.955             0   
940      6   30   attempted           1  ...   2130.0     −1.96             0   
941     10   28   attempted           1  ...   1930.0     −1.86             0   
942      1   11  conspiracy           1  ...   4350.0     −3.17             0   

     pre_coup  pre_coup5  pre_coup_fail  pre_coup_succ  pre_leader_harm  \
0           7          0              1              6                1   
1          14          0              6              8                2   
2          10          0              6              4                1   
3          11          0              7              4                0   
4          12          1              7              5                0   
..        ...        ...            ...            ...              ...   
938         1          1              1              0                0   
939         2          0              2              0                0   
940         3          1              3              0                0   
941         4          0              4              0                0   
942         0          0              0              0                0   

     pre_leader_court  standing_regime  
0                   2                1  
1                   0                1  
2                   0                1  
3                   1                1  
4                   3                1  
..                ...              ...  
938                 0                1  
939                 0                1  
940                 0                1  
941                 0                1  
942                 0                1  

[943 rows x 76 columns]

coup_id  cowcode     country  year  democracy  autocracy  \
39   70304082010      703  Kyrgyzstan  2010      -88.0      -88.0   
96   16006161955      160   Argentina  1955      -88.0      -88.0   
97   16009161955      160   Argentina  1955      -88.0      -88.0   
98   16011131955      160   Argentina  1955      -88.0      -88.0   
99   16006091956      160   Argentina  1956      -88.0      -88.0   
..           ...      ...         ...   ...        ...        ...   
852  80010141973      800    Thailand  1973      -88.0      -88.0   
854  80003261977      800    Thailand  1977      -88.0      -88.0   
855  80010201977      800    Thailand  1977      -88.0      -88.0   
881  64005271960      640      Turkey  1960      -88.0      -88.0   
888  50004151966      500      Uganda  1966      -88.0      -88.0   

     polity2_score  years_of_regime_pow  exec_competitive_score  \
39             4.0                  0.0                   -88.0   
96            -6.0                  0.0                   -88.0   
97            -6.0                  0.0                   -88.0   
98            -6.0                  0.0                   -88.0   
99            -2.0                  0.0                   -88.0   
..             ...                  ...                     ...   
852           -2.0                  0.0                   -88.0   
854           -2.0                  0.0                   -88.0   
855           -2.0                  0.0                   -88.0   
881            7.0                  0.0                   -88.0   
888            0.0                  0.0                   -88.0   

     exec_openness_score  exec_constraint_score  political_regularity  \
39                 -88.0                  -88.0                 -88.0   
96                 -88.0                  -88.0                 -88.0   
97                 -88.0                  -88.0                 -88.0   
98                 -88.0                  -88.0                 -88.0   
99                 -88.0                  -88.0                 -88.0   
..                   ...                    ...                   ...   
852                -88.0                  -88.0                 -88.0   
854                -88.0                  -88.0                 -88.0   
855                -88.0                  -88.0                 -88.0   
881                -88.0                  -88.0                 -88.0   
888                -88.0                  -88.0                 -88.0   

     political_competitiveness  exec_recruitment  
39                       -88.0             -88.0  
96                       -88.0             -88.0  
97                       -88.0             -88.0  
98                       -88.0             -88.0  
99                       -88.0             -88.0  
..                         ...               ...  
852                      -88.0             -88.0  
854                      -88.0             -88.0  
855                      -88.0             -88.0  
881                      -88.0             -88.0  
888                      -88.0             -88.0  

[61 rows x 14 columns]

Syria                               7
Portugal                            6
Argentina                           5
Thailand                            3
Dominican Republic                  3
Philippines                         3
Haiti                               2
El Salvador                         2
Hungary                             2
German Democratic Republic          2
Fiji                                2
Mali                                2
Democratic Republic of the Congo    2
Cuba                                2
Burundi                             2
Brazil                              2
Senegal                             1
Rumania                             1
Turkey                              1
Sudan                               1
Kyrgyzstan                          1
Peru                                1
Liberia                             1
Italy                               1
Guinea-Bissau                       1
Ecuador                             1
Cyprus      

In [5]:
# Impute 0s for -66 and create interregum (b/t governments) column 

coup_curr_ver["standing_regime"].mask(coup_curr_ver['democracy'] == -66, 0, inplace=True)
for feature in polity_imputation_cols:
    coup_curr_ver[feature].mask(coup_curr_ver[feature] == -66, 0, inplace=True)
#display(coup_curr_ver)

coup_curr_ver.to_csv("-77_yeeted.csv")


#Impute value from nearest neighbor for missing value at row with -88 (rationale: -88 indicates a transition period)

relevant_p5 = p5_df[["country"] + polity_old_cols]

coup_curr_ver["standing_regime"].mask(coup_curr_ver['democracy'] == -88, 0, inplace=True)

country_list = [x for x in coup_curr_ver['country'].unique() if x in relevant_p5['country'].unique()]

for country in country_list:
    country_df = relevant_p5[relevant_p5['country'] == country]

    for i in range(len(polity_imputation_cols)):
        feature_avg = int(np.round(np.mean(country_df[country_df[polity_old_cols[i]] != -88][polity_old_cols[i]])))
        
        # Impute means of non-standing government and missing observation
        coup_curr_ver[polity_imputation_cols[i]].mask(coup_curr_ver[polity_imputation_cols[i]] == -88, feature_avg, inplace=True)

        coup_curr_ver[polity_imputation_cols[i]].mask(coup_curr_ver[polity_imputation_cols[i]].isna(), feature_avg, inplace=True)

coup_curr_ver.to_csv("coup_merged_v3.csv")

y = coup_curr_ver.isna().sum(axis=0)
display(coup_curr_ver.isna().sum(axis=0))
y[y > 0]       








Unnamed: 0.1        0
Unnamed: 0          0
coup_id             0
cowcode             0
country             0
                   ..
pre_coup_fail       0
pre_coup_succ       0
pre_leader_harm     0
pre_leader_court    0
standing_regime     0
Length: 76, dtype: int64

indep_viol_mag                 67
intl_viol_mag                  67
intl_war_mag                   67
civil_viol_mag                 67
civil_war_mag                  67
ethnic_viol_mag                67
ethnic_war_mag                 67
total_intl_viol_mag            70
total_civil_viol_mag           70
neighbors_w_intl_viol          70
neighbors_w_civil_viol         70
neighbors_in_region            67
regional_intl_viol_mag         68
regional_civil_viol_mag        68
total_regional_viol_mag        68
reg_states_intl_viol_count     68
reg_states_civil_viol_count    68
pop_growth                      1
dtype: int64

In [6]:
# Drop 2012 poly-vio and merge 2018 poly-vio into csv

coup_curr_ver = coup_curr_ver.drop(columns=['indep_viol_mag', 'intl_viol_mag', 'intl_war_mag',
       'civil_viol_mag', 'civil_war_mag', 'ethnic_viol_mag', 'ethnic_war_mag',
       'total_intl_viol_mag', 'total_civil_viol_mag', 'neighbors_w_intl_viol',
       'neighbors_w_civil_viol', 'neighbors_in_region',
       'regional_intl_viol_mag', 'regional_civil_viol_mag',
       'total_regional_viol_mag', 'reg_states_intl_viol_count',
       'reg_states_civil_viol_count'])

#Grenada, the Maldives, Seychelles, Sao Tome and Principe, Dominica, are not present, Egypt(1947) is not present

alt_code_dict = {769:770, 365:393, 364:365, 529:530, 626:625, 525:626}

# for key,val in alt_code_dict.items():
#        poly_vio_df['ccode'] = 

poly_vio_df['ccode'] = poly_vio_df['ccode'].replace(alt_code_dict)

display(poly_vio_df[poly_vio_df['ccode'] == 770])

poly_vio_df['cowcode'] = poly_vio_df.ccode

relevant_polyv_cols = ['cowcode', 'year', 'intind',
       'intviol', 'intwar', 'civviol', 'civwar', 'ethviol', 'ethwar', 'inttot',
       'civtot', 'actotal', 'nborder']
new_coup = pd.merge(coup_curr_ver, poly_vio_df[relevant_polyv_cols], how='left', on=['cowcode', 'year'])

missing = new_coup[new_coup.civtot.isna() == True]

fill_list = []

for i in range(len(missing)):
       curr_code = new_coup[new_coup.civtot.isna() == True].iloc[i]['cowcode']
       curr_year = new_coup[new_coup.civtot.isna() == True].iloc[i]['year']
       
       new_entry = poly_vio_df[(poly_vio_df['cowcode'] == curr_code) & (poly_vio_df['year'] == curr_year-1)][relevant_polyv_cols]
       
       if new_entry.empty:
              new_entry = poly_vio_df[(poly_vio_df['cowcode'] == curr_code) & (poly_vio_df['year'] == curr_year+1)][relevant_polyv_cols]
       if new_entry.empty:
              continue
       fill_list.append(new_entry.values.tolist())

       # change = new_coup['civviol'].isna()
       # new_coup.loc[change, [relevant_polyv_cols[2:]]] = []
       # display(new_coup.loc[new_coup.civtot.isna() == True])

new_coup.rename(columns={'intind':'indep_viol_mag', 'intviol':'intl_viol_mag',
       'intwar':'intl_war_mag', 'civviol':'civil_viol_mag', 'civwar':'civil_war_mag', 'ethviol':'ethnic_viol_mag',
       'ethwar':'ethnic_war_mag', 'inttot':'total_intl_viol_mag', 'civtot':'total_civil_viol_mag',
       'actotal': 'total_viol_mag' , 'nborder':'number_of_neighbors'})

y2 = new_coup[new_coup.civtot.isna() == True]
display(y2)




scode  ccode   country  year  ind  intind  intviol  intwar  civviol  \
6658   PAK    770  Pakistan  1971    1     0.0      0.0     3.0      0.0   
6659   PAK    770  Pakistan  1972    1     0.0      0.0     0.0      0.0   
6660   PAK    770  Pakistan  1973    1     0.0      0.0     0.0      0.0   
6661   PAK    770  Pakistan  1974    1     0.0      0.0     0.0      0.0   
6662   PAK    770  Pakistan  1975    1     0.0      0.0     0.0      0.0   
...    ...    ...       ...   ...  ...     ...      ...     ...      ...   
7018   PKS    770  Pakistan  1966    1     0.0      0.0     0.0      0.0   
7019   PKS    770  Pakistan  1967    1     0.0      0.0     0.0      0.0   
7020   PKS    770  Pakistan  1968    1     0.0      0.0     0.0      0.0   
7021   PKS    770  Pakistan  1969    1     0.0      0.0     0.0      0.0   
7022   PKS    770  Pakistan  1970    1     0.0      0.0     0.0      0.0   

      civwar  ethviol  ethwar  inttot  civtot  actotal  nborder  region  \
6658     0.0      0.0     6.0     3.0     6.0      9.0      4.0     6.0   
6659     0.0      0.0     0.0     0.0     0.0      0.0      4.0     6.0   
6660     0.0      0.0     2.0     0.0     2.0      2.0      4.0     6.0   
6661     0.0      0.0     2.0     0.0     2.0      2.0      4.0     6.0   
6662     0.0      0.0     2.0     0.0     2.0      2.0      4.0     6.0   
...      ...      ...     ...     ...     ...      ...      ...     ...   
7018     0.0      0.0     0.0     0.0     0.0      0.0      5.0     6.0   
7019     0.0      0.0     0.0     0.0     0.0      0.0      5.0     6.0   
7020     0.0      0.0     0.0     0.0     0.0      0.0      5.0     6.0   
7021     0.0      0.0     0.0     0.0     0.0      0.0      5.0     6.0   
7022     0.0      0.0     0.0     0.0     0.0      0.0      5.0     6.0   

      nregion  
6658     10.0  
6659     10.0  
6660     10.0  
6661     10.0  
6662     10.0  
...       ...  
7018      9.0  
7019      9.0  
7020      9.0  
7021      9.0  
7022      9.0  

[73 rows x 18 columns]

Unnamed: 0.1  Unnamed: 0      coup_id  cowcode  \
15             15          15  78102072012      781   
25             25          25  61504022019      615   
28             28          28  48101072019      481   
32             32          32  62504112019      625   
33             33          33  61601142011      616   
34             34          34  10101232019      101   
35             35          35  10104302019      101   
52             51          51  49012302013      480   
179           178         178  14010291945      140   
309           308         308   5412191981       54   
335           334         334  65112231947      651   
402           401         401   5503131979       55   
403           402         402   5510141983       55   
404           403         403   5510191983       55   
405           404         404   5510311983       55   
572           571         571  78104261980      781   
573           572         572  78109101983      781   
574           573         573  78111031988      781   
742           741         741  40308151995      403   
743           742         742  40307162003      403   
745           744         744  34010071999      340   
746           745         745  59106051977      591   
747           746         746  59111161979      591   
748           747         747  59111251981      591   
776           775         775  62508181955      625   
907           906         906  36508211991      365   

                              country  year  month  day  event_type  \
15                           Maldives  2012      2    7        coup   
25                            Algeria  2019      4    2        coup   
28                              Gabon  2019      1    7   attempted   
32                              Sudan  2019      4   11        coup   
33                            Tunisia  2019      1   14        coup   
34                          Venezuela  2019      1   23   attempted   
35                          Venezuela  2019      4   30   attempted   
52   Democratic Republic of the Congo  2013     12   30   attempted   
179                            Brazil  1945     10   29        coup   
309                          Dominica  1981     12   19   attempted   
335                             Egypt  1947     12   23  conspiracy   
402                           Grenada  1979      3   13        coup   
403                           Grenada  1983     10   14        coup   
404                           Grenada  1983     10   19        coup   
405                           Grenada  1983     10   31        coup   
572                          Maldives  1980      4   26   attempted   
573                          Maldives  1983      9   10  conspiracy   
574                          Maldives  1988     11    3   attempted   
742             Sao Tome and Principe  1995      8   15        coup   
743             Sao Tome and Principe  2003      7   16        coup   
745                            Serbia  1999     10    7        coup   
746                        Seychelles  1977      6    5        coup   
747                        Seychelles  1979     11   16  conspiracy   
748                        Seychelles  1981     11   25   attempted   
776                             Sudan  1955      8   18   attempted   
907                              USSR  1991      8   21        coup   

     unrealized  ...  intviol  intwar  civviol  civwar  ethviol  ethwar  \
15            0  ...      NaN     NaN      NaN     NaN      NaN     NaN   
25            0  ...      NaN     NaN      NaN     NaN      NaN     NaN   
28            1  ...      NaN     NaN      NaN     NaN      NaN     NaN   
32            0  ...      NaN     NaN      NaN     NaN      NaN     NaN   
33            0  ...      NaN     NaN      NaN     NaN      NaN     NaN   
34            1  ...      NaN     NaN      NaN     NaN      NaN     NaN   
35            1  ...      NaN     NaN      NaN     NaN      NaN     NaN   
5

In [7]:

code_list = [x[0][0] for x in fill_list]
year_list = [x[0][1] for x in fill_list]

print(code_list)
print(year_list)


change = new_coup['civviol'].isna()
for i in range(len(fill_list)):
    new_coup.loc[change & (new_coup['cowcode'] == code_list[i]) & (new_coup['year'] == year_list[i]+1), relevant_polyv_cols[2:]] = fill_list[i][0][2:]
    new_coup.loc[change & (new_coup['cowcode'] == code_list[i]) & (new_coup['year'] == year_list[i]-1), relevant_polyv_cols[2:]] = fill_list[i][0][2:]
 
# Switched cow code so DRC must be entered manually
new_coup.loc[change & (new_coup['cowcode'] == 480) & (new_coup['year'] == 2013), relevant_polyv_cols[2:]]= [0,0,0,0,5,1,0,0,6,6,8]

#The other 16 columns cannot be resolved and so it is we could consider dropping them

y2 = new_coup[new_coup.civtot.isna() == True]
display(y2)

# new_coup = new_coup.drop(index=[15,309,335,402,403,404,405,572,573,574,742,743,745,746,747,748])

new_coup[new_coup['pop_growth'].isna() == True]['pop_growth'] = 1.35

y3 = coup_curr_ver['pop_growth'].isna().sum()
display(coup_curr_ver.isna().sum(axis=0))
y3[y3 > 0]

new_coup.to_csv("coup_no_missingness.csv")



[615.0, 481.0, 625.0, 616.0, 101.0, 101.0, 140.0, 625.0, 365.0]
[2018.0, 2018.0, 2018.0, 2018.0, 2018.0, 2018.0, 1946.0, 1956.0, 1990.0]


Unnamed: 0.1  Unnamed: 0      coup_id  cowcode                country  \
15             15          15  78102072012      781               Maldives   
309           308         308   5412191981       54               Dominica   
335           334         334  65112231947      651                  Egypt   
402           401         401   5503131979       55                Grenada   
403           402         402   5510141983       55                Grenada   
404           403         403   5510191983       55                Grenada   
405           404         404   5510311983       55                Grenada   
572           571         571  78104261980      781               Maldives   
573           572         572  78109101983      781               Maldives   
574           573         573  78111031988      781               Maldives   
742           741         741  40308151995      403  Sao Tome and Principe   
743           742         742  40307162003      403  Sao Tome and Principe   
745           744         744  34010071999      340                 Serbia   
746           745         745  59106051977      591             Seychelles   
747           746         746  59111161979      591             Seychelles   
748           747         747  59111251981      591             Seychelles   

     year  month  day  event_type  unrealized  ...  intviol  intwar  civviol  \
15   2012      2    7        coup           0  ...      NaN     NaN      NaN   
309  1981     12   19   attempted           1  ...      NaN     NaN      NaN   
335  1947     12   23  conspiracy           1  ...      NaN     NaN      NaN   
402  1979      3   13        coup           0  ...      NaN     NaN      NaN   
403  1983     10   14        coup           0  ...      NaN     NaN      NaN   
404  1983     10   19        coup           0  ...      NaN     NaN      NaN   
405  1983     10   31        coup           0  ...      NaN     NaN      NaN   
572  1980      4   26   attempted           1  ...      NaN     NaN      NaN   
573  1983      9   10  conspiracy           1  ...      NaN     NaN      NaN   
574  1988     11    3   attempted           1  ...      NaN     NaN      NaN   
742  1995      8   15        coup           0  ...      NaN     NaN      NaN   
743  2003      7   16        coup           0  ...      NaN     NaN      NaN   
745  1999     10    7        coup           0  ...      NaN     NaN      NaN   
746  1977      6    5        coup           0  ...      NaN     NaN      NaN   
747  1979     11   16  conspiracy           1  ...      NaN     NaN      NaN   
748  1981     11   25   attempted           1  ...      NaN     NaN      NaN   

     civwar  ethviol  ethwar  inttot  civtot  actotal  nborder  
15      NaN      NaN     NaN     NaN     NaN      NaN      NaN  
309     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
335     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
402     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
403     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
404     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
405     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
572     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
573     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
574     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
742     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
743     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
745     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
746     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
747     NaN      NaN     NaN     NaN     NaN      NaN      NaN  
748     NaN      NaN     NaN     NaN     NaN      NaN      NaN  

[16 rows x 70 columns]

C:\Users\angle\AppData\Local\Temp\ipykernel_22124\2729892151.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_coup[new_coup['pop_growth'].isna() == True]['pop_growth'] = 1.35


Unnamed: 0.1                 0
Unnamed: 0                   0
coup_id                      0
cowcode                      0
country                      0
year                         0
month                        0
day                          0
event_type                   0
unrealized                   0
realized                     0
conspiracy                   0
attempt                      0
military                     0
dissident                    0
rebel                        0
palace                       0
foreign                      0
auto                         0
resign                       0
popular                      0
counter                      0
other                        0
noharm                       0
injured                      0
killed                       0
harrest                      0
jailed                       0
tried                        0
fled                         0
exile                        0
region                       0
sub-regi